# ユーザベースの協調フィルタリング

### 評価値データベース

- データを pandas DataFrame として格納
  - ユーザ名、アイテム名を指定して値を参照

![24](slides/24-table.png)

- 「あなた」はアイテム5にどのような評価を与えるかを予測
  - 評価値は5段階
  - 高い評価を与えると予測されるならば、推薦リストに加えるべき
- 「あなた」に類似した嗜好を持つユーザは誰か？

In [ ]:
import numpy as np
import pandas as pd

# リストで元データを作成
data1 = [
    [5, 3, 4, 4],     # You
    [3, 1, 2, 3, 3],  # User1
    [4, 3, 4, 3, 5],  # User2
    [3, 3, 1, 5, 4],  # User3
    [1, 5, 5, 2, 1]   # User4
]

data2 = [
    [5, 3, 4, 4],     # You
    [3, 1, 2, 3, 3],  # User1
    [4, 3, 4, 3, 5],  # User2
    [3, 3, 1, 5, 4],  # User3
    [5, np.nan, np.nan, np.nan, 1]   # User4
]

# pandas DataFrame を上記の data1, data2 から作成
# - columns: 列ラベルの指定
# - index: 行ラベルの指定
db1 = pd.DataFrame(data1,
                   columns=['Item1', 'Item2', 'Item3', 'Item4', 'Item5'],
                   index=['You', 'User1', 'User2', 'User3', 'User4'])

db2 = pd.DataFrame(data2,
                   columns=['Item1', 'Item2', 'Item3', 'Item4', 'Item5'],
                   index=['You', 'User1', 'User2', 'User3', 'User4'])

# 確認
# - 'You', 'Item5' は値が無いため NaN (np.nan) 
# - 'Item5' 列は 'You' が NaN（浮動小数点数）のため、他の値も浮動小数点数
db1

### 類似度を計算する関数の定義

- ユークリッド距離（1を足して値の逆数）
- 相関係数

類似度を計算するためには、評価が共通にあるアイテムのみを取り出す必要

<hr>

![19](slides/19.png)

In [ ]:
# ユークリッド距離（1を足して値の逆数）
from scipy.spatial.distance import euclidean

def similarity_euclide(df, l1, l2):
    a = df.loc[[l1, l2]].dropna(axis=1)
    # 共通に評価があるアイテムが1つだけの場合は除外
    # - NaN が返される
    if len(a.iloc[0]) < 2:
        return np.nan
    return 1 / (1 + euclidean(a.iloc[0], a.iloc[1]))

similarity_euclide(db1, 'You', 'User1')
# - NaN が返される例
#similarity_euclide(db2, 'You', 'User4')

In [ ]:
# 相関係数
# - 共通に評価があるアイテムが1つだけの場合は NaN が返される
def similarity_correlation(df, l1, l2):
    _df = df.loc[[l1, l2]].dropna(axis=1).T
    return _df.corr().iloc[0, 1]

similarity_correlation(db1, 'You', 'User1')
# - NaN が返される例
#similarity_correlation(db2, 'You', 'User4')

![21](slides/21.png)

### 近傍ユーザの類似度の計算

- 平均評価値を求める
- 'You' と他ユーザとの間の類似度（相関係数）を求める

In [ ]:
# 平均評価値の計算
mean = db1.mean(axis=1)
# 確認
mean

In [ ]:
# 平均評価値列の追加
db1['mean'] = mean
# 類似度列を追加し、NaN で初期化
db1['similarity'] = np.nan

# 各ユーザについて類似度を計算し、DataFrameに代入
for u in db1.index.drop('You'):
    db1.at[u, 'similarity'] = similarity_correlation(db1, u, 'You')

# 確認
db1

### 評価の予測

- 相関係数の場合は正の相関を持つユーザのみ
- 類似ユーザの評価値の加重平均を計算

In [ ]:
db1_sim = db1.query('similarity > 0')
db1_sim

In [ ]:
# 加重平均を計算
avg = ((db1_sim['Item5'] - db1_sim['mean']) * db1_sim.similarity).sum()
avg = avg / db1_sim.similarity.sum()
# 平均評価値に加算
db1.at['You', 'mean'] + avg